In [166]:
val test_input="""162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
"""

In [173]:
import java.nio.file.Files
import java.nio.file.Paths

val use_input = true
var numConnections = 10
val debug = false
val seq = if (use_input) {
    numConnections = 1000
    Files.readString(Paths.get("input.txt"))
} else {
    numConnections = 10
    test_input
}.trim().split('\n').toList()

In [168]:
data class Vec3(val x:Double, val y:Double, val z:Double)
typealias NodeID = Int
typealias ConnectionID = Int

val vectors = seq.map {
    val s = it.split(",")
    Vec3(s[0].toDouble(), s[1].toDouble(), s[2].toDouble())
}

In [169]:
val distances = mutableMapOf<Double, Pair<NodeID,NodeID>>()

for (i in 0 until seq.size) {
    for (j in i+1 until seq.size) {
        val a = vectors[i]
        val b = vectors[j]
        val dist = sqrt((a.x-b.x).pow(2.0) + (a.y-b.y).pow(2.0) + (a.z-b.z).pow(2.0))
        distances[dist] = Pair(i,j)
    }
}

In [174]:
//maps from node to connectionsID
val connections = mutableMapOf<NodeID, ConnectionID>()
var connectionId = 0
var lastConnection : Pair<NodeID, NodeID>? = null
var connectionCount = 0
var result = 0
distances.toSortedMap().entries.forEachIndexed { id, it ->
    val nodeA : NodeID = it.value.first
    val nodeB = it.value.second
    val nodeAConnectionId : ConnectionID? = connections[nodeA]
    val nodeBConnectionId : ConnectionID? = connections[nodeB]

    if (nodeAConnectionId == null && nodeBConnectionId == null) {
        //create a new connection
        connections[nodeA] = connectionId
        connections[nodeB] = connectionId
        if (debug) println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> new circuit $connectionId: $nodeA -> $nodeB")
        connectionId++
    } else if (nodeBConnectionId == null && nodeAConnectionId!=null) {
        //
        connections[nodeB] = nodeAConnectionId
        if (debug) println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> $nodeB joining circuit $nodeAConnectionId")
        lastConnection = Pair(nodeA, nodeB)
    } else if (nodeAConnectionId == null && nodeBConnectionId!=null) {
        connections[nodeA] = nodeBConnectionId
        if (debug) println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> $nodeA joining circuit $nodeBConnectionId")
        lastConnection = Pair(nodeA, nodeB)
    } else if (nodeAConnectionId != null && nodeBConnectionId!=null && nodeAConnectionId != nodeBConnectionId){
        val minConnectionID = if (nodeAConnectionId < nodeBConnectionId) nodeAConnectionId else nodeBConnectionId
        if (debug) println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> joining circuits $nodeAConnectionId and $nodeBConnectionId to $minConnectionID")
        connections.entries.forEach { (n,c) ->
            if (c == nodeAConnectionId) connections[n] = minConnectionID
            if (c == nodeBConnectionId) connections[n] = minConnectionID
        }
        lastConnection = Pair(nodeA, nodeB)
    } else if (nodeAConnectionId == nodeBConnectionId){
        //nothing, already connected
        //println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> already connected")
    } else {
        println("connection $id: $nodeA(${nodeAConnectionId}) and $nodeB(${nodeBConnectionId}) -> WTF")
    }
    connectionCount++
    if (connectionCount == numConnections) {
        result = connections.values.groupingBy { it }.eachCount().entries.sortedByDescending { it.value }.take(3).fold(1) { acc, (value, count) -> acc * count}


    }
    //println(connections)
}
println("top three circuits in ${numConnections} connections multiplied: $result")
println("last connection joining circuits: $lastConnection")
val m = vectors[lastConnection!!.first].x.toLong() * vectors[lastConnection!!.second].x.toLong()
println("x of last two circuits multiplied: $m")

top three circuits in 1000 connections multiplied: 50760
last connection joining circuits: (848, 952)
x of last two circuits multiplied: 3206508875
